<a href="https://colab.research.google.com/github/SaeedM75/Deep-learning/blob/main/hw1p2_Saeed_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
!pip install --upgrade --force-reinstall --no-deps kaggle
#!pip install ipdb
!pip install wandb -q
torch.backends.cudnn.benchmark = True
!pip install tqdm


     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=cc477aa8d3915f90a525a3cb133513f928bc960104c257032477a75107bc1d99
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 1.7 MB 8.2 MB/s 
     |████████████████████████████████| 180 kB 15.5 MB/s 
     |████████████████████████████████| 144 kB 39.7 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 


In [ ]:
import wandb
# from tqdm import tqdm
from tqdm.notebook import tqdm, trange
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive  
drive.mount('/content/gdrive')

!cp '/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW1/part 2/11-785-s22-hw1p2.zip' '/content'
!cp '/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW1/part 2/data/train_filenames_subset_8192_v2.csv' '/content'
!cp '/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW1/part 2/data/train_filenames_subset_2048_v2.csv' '/content'
!unzip /content/11-785-s22-hw1p2.zip

In [ ]:
!mkdir '/root/.kaggle'
import json
token = {"username":"smoayedp","key":"42e16d25f2ea76fd2f8b490568d200ba"}
!pwd
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(token, file)

for i in range(2):
  !chmod 600 '/root/.kaggle/kaggle.json'
  !cp '/root/.kaggle/kaggle.json' ~/.kaggle/
  !kaggle config set -n path -v /root/
  !cat '/root/.kaggle/kaggle.json'

/content
cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root/
{
  "username": "smoayedp",
  "key": "42e16d25f2ea76fd2f8b490568d200ba",
  "path": "/root/"
}cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
- path is now set to: /root/
{
  "username": "smoayedp",
  "key": "42e16d25f2ea76fd2f8b490568d200ba",
  "path": "/root/"
}

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # TODO: Please try different architectures
        # in_size = 13
        context = 20
        in_size = 13 * (1 + 2*context)  
        
        layers = [
            nn.Linear(in_size, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p = 0.1),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(p = 0.1),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            
            nn.Linear(256, 40)

        ]
        
        # layers = [
        #     nn.Linear(in_size, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 64),
        #     nn.ReLU(),
        #     nn.Linear(64, 40),

        # ]

        self.layers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.layers(A0)
        return x

In [ ]:

class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition +"/transcript/"
        
        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]
            
              
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
        
        assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]
            
            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))
            # import ipdb, ipdb.set_trace
            
        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y
    
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context = 0):
        assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        self.context = context

        if context == 0:
            
            self.X, self.Y = X, Y
            # import ipdb; ipdb.set_trace()
        else:
            # c = np.zeros((context, 13))
            # self.X = np.concatenate([c,X,c], axis = 0)
    
            X = np.pad(X, ((context,context), (0,0)), 'constant', constant_values=(0,0))
            self.X, self.Y = X, Y

        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            xx = self.X[i:(i + 2*self.context + 1)].flatten()
            yy = self.Y[i]
            # TODO xx, yy = ...
            # pass
        return xx, yy
    



In [ ]:
class test_LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle= False, partition="test-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        # self.Y_dir = data_path + "/" + partition +"/transcript/"
        
        self.X_names = os.listdir(self.X_dir)
        

        # using a small part of the dataset to debug
        if csvpath:
            self.X_names = list(pd.read_csv(csvpath).file)
              
        
        # assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
          sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
          
          X, Y = [], []
          for j in sample_range:
              X_path = self.X_dir + self.X_names[j]
              # Y_path = self.Y_dir + self.Y_names[j]
              
              # label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

              X_data = np.load(X_path)
              X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
              X.append(X_data)
              # Y.append(np.array(label))
              # import ipdb, ipdb.set_trace
              
          X = np.concatenate(X)
          return X
    
class test_LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y = None, context = 0):
        # assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        self.context = context

        if context == 0:
            
            self.X = X
            # import ipdb; ipdb.set_trace()
        else:
            # c = np.zeros((context, 13))
            # self.X = np.concatenate([c,X,c], axis = 0)
    
            X = np.pad(X, ((context,context), (0,0)), 'constant', constant_values=(0,0))
            self.X = X

        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            # yy = self.Y[i]
        else:
            xx = self.X[i:(i + 2*self.context + 1)].flatten()
            # yy = self.Y[i]
            # TODO xx, yy = ...
            # pass
        return xx

In [ ]:
from torch.cuda.amp import autocast, GradScaler 

def train(args, model, device, train_samples, optimizer, criterion, epoch):
    model.train()
    for i in range(len(train_samples)) :
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True)
        
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Train Epoch {epoch}")
            for data, target in tepoch:

        # for batch_idx, (data, target) in enumerate(train_loader):
                data = data.float().to(device)
                target = target.long().to(device)

                optimizer.zero_grad()
                
                with autocast():
                    output = model(data)
                    loss = criterion(output, target)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                tepoch.set_postfix(loss=loss.item())

            # output = model(data)
            # loss = criterion(output, target)
            # loss.backward()
            # optimizer.step()
            # prog_bar.set_postfix(loss = f'{loss.item():.4f}' )
            
            # if batch_idx % args['log_interval'] == 0:
            #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #         epoch, batch_idx * len(data), len(train_loader.dataset),
            #         100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, dev_samples):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)) :
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)
            with tqdm(test_loader, unit="batch") as vepoch:
                vepoch.set_description(f"Eval")
                for data, true_y in vepoch:
                    data = data.float().to(device)
                    true_y = true_y.long().to(device)                
                    
                    output = model(data)
                    pred_y = torch.argmax(output, axis=1)
                    pred_y_list.extend(pred_y.tolist())
                    true_y_list.extend(true_y.tolist())
                    vepoch.set_postfix(Predicting= "Ongoing")
    
    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    wandb.log({"Valid Accuracy": train_accuracy, "Epoch": epoch, "learning_rate": optimizer.state_dict()['param_groups'][0]['lr']})
    return train_accuracy




def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = Network().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3, mode='min')
    scaler = GradScaler()
    criterion = torch.nn.CrossEntropyLoss()
    # If you want to use full Dataset, please pass None to csvpath
    train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="hw1p2_student_data/train-clean-100", csvpath="/content/train_filenames_subset_8192_v2.csv")
    dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="hw1p2_student_data/dev-clean")

    for epoch in range(1, args['epoch'] + 1):
        train(args, model, device, train_samples, optimizer, criterion, epoch)
        test_acc = test(args, model, device, dev_samples)
        scheduler.step(test_acc)
        print('Dev accuracy ', test_acc)


if __name__ == '__main__':
    args = {
        'batch_size': 2048,
        'context': 0,
        'log_interval': 200,
        'LIBRI_PATH': '/content',
        'lr': 0.001,
        'epoch': 3
    }
    # main(args)

In [ ]:
a  = 2


In [ ]:
import time


args = {
        'batch_size': 4096,
        'context':20,
        'log_interval': 200,
        'LIBRI_PATH': '/content',
        'lr': 0.001,
        'epoch': 20
    }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Network().to(device)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = Network().to(device)
# model.load_state_dict(torch.load('/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW1/part 2/'+
#                'epoch_'+ str(10)+'.pth'))

wandb.init(project="high cutoff",name='dropout_0.2',config={'lr':0.001,'bn':'true','num_layers':4, 'model architecture': model.parameters})
wandb.watch(model, log="all")



optimizer = optim.Adam(model.parameters(), lr=args['lr'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3, mode='min')
scaler = GradScaler()
criterion = torch.nn.CrossEntropyLoss()
# prog_bar = tqdm()
# If you want to use full Dataset, please pass None to csvpath
# train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="hw1p2_student_data/train-clean-100", csvpath="/content/train_filenames_subset_2048_v2.csv")
train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="hw1p2_student_data/train-clean-100", csvpath=None)
dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="hw1p2_student_data/dev-clean")

time_list = []
for epoch in range(1, args['epoch'] + 1):
    t1 = time.time()
    train(args, model, device, train_samples, optimizer, criterion, epoch)
    test_acc = test(args, model, device, dev_samples)
    scheduler.step(test_acc)
    print('learning rate ', optimizer.state_dict()['param_groups'][0]['lr'])
    print('Dev accuracy ', test_acc)
    t2 = time.time()
    time_list.append(t2 - t1)
    torch.save(model.state_dict(), '/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW1/part 2/'+
               'epoch_'+ str(epoch)+'.pth')

wandb.finish()
# print(np.mean(time_list))

  # print(id, doc)

Epoch,▁
Valid Accuracy,▁
learning_rate,▁
Epoch,1
Valid Accuracy,0.80301
learning_rate,0.001


  0%|          | 0/6187 [00:00<?, ?batch/s]

  0%|          | 0/2650 [00:00<?, ?batch/s]

  0%|          | 0/474 [00:00<?, ?batch/s]

learning rate  0.001
Dev accuracy  0.8068506980143443


...

  0%|          | 0/6187 [00:00<?, ?batch/s]

  0%|          | 0/2650 [00:00<?, ?batch/s]

  0%|          | 0/474 [00:00<?, ?batch/s]

learning rate  1e-05
Dev accuracy  0.8405240578561195


  0%|          | 0/6187 [00:00<?, ?batch/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Network().to(device)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/CMU PhD semester/Sem 8/Deep Learning/HWs/HW1/part 2/'+
               'epoch_'+ str(12)+'.pth', map_location=torch.device('cpu')))
 
args = {
        'batch_size': 4096,
        'context':20,
        'log_interval': 200,
        'LIBRI_PATH': '/content',
        'lr': 0.001,
        'epoch': 20
    }

In [ ]:

test_samples = test_LibriSamples(data_path = args['LIBRI_PATH'], shuffle=False, partition="hw1p2_student_data/test-clean", csvpath="/content/test_order.csv" )

# print(list(test_samples))

model.eval()
true_y_list = []
pred_y_list = []
with torch.no_grad():
    for i in range(len(test_samples)):
        X = test_samples[i]
        test_items = test_LibriItems(X, context=args['context'])
        test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

        for data in test_loader:
            
            data = data.float().to(device)
            # true_y = true_y.long().to(device)                
            # print(len(data))
            output = model(data)
            # print(len(output))
            pred_y = torch.argmax(output, axis=1)
            pred_y_list.extend(pred_y.tolist())
            # true_y_list.extend(true_y.tolist())



In [ ]:
pd_res = pd.DataFrame(pred_y_list, columns= ["label"])
pd_res.index.names = ['id']
pd_res.to_csv("submission.csv")

In [ ]:
a = np.zeros((10, ))
b = np.ones((1000, ))
b = np.concatenate([a,b,a], axis = 0)
b.shape

(1020,)

1.0000000000000002e-06

In [ ]:
!nvidia-smi

Thu Feb  3 17:19:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    58W / 149W |    557MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
type(optimizer.state_dict()['param_groups'][0]['lr'])

float